## import

In [1]:
import sys
import math
import argparse
import numpy as np
import matplotlib.pyplot as plt
from scipy.special import expit
from numpy.linalg import det, inv

# ----------------------------------------------
# test case 1 : default                    
# test case 2 : -x1 1 2 -y1 1 2 -x2 3 4 -y2 3 4                         
# ----------------------------------------------

## passing arguments

In [2]:
def get_parser(test_case):
    parser = argparse.ArgumentParser()
    parser.add_argument('-N', type = int, default=50)
    parser.add_argument('-x1', nargs=2, type=float, default=[1.0, 2.0])  #m_x1, v_x1
    parser.add_argument('-y1', nargs=2, type=float, default=[1.0, 2.0])  #m_y1, v_y1
    parser.add_argument('-x2', nargs=2, type=float, default=[10.0, 2.0]) #m_x2, v_x2
    parser.add_argument('-y2', nargs=2, type=float, default=[10.0, 2.0]) #m_y2, v_y2
    
    return parser.parse_args(args = test_case)

## data generator

In [3]:
def data_generator_univar(m, s):
    Z = sum(np.random.uniform(0, 1, 12)) - 6
    return(m + math.sqrt(s) * Z )


def data_generator(x, y, N):
    # X = [x, y, 1] 
    # since f(x, y) = ax + by + c = Xw
    data = np.zeros((N, 3))
    mx, vx = x[0], x[1]
    my, vy = y[0], y[1]
    for i in range(N):
        data[i, 0] = data_generator_univar(mx, vx)
        data[i, 1] = data_generator_univar(my, vy)
        data[i, 2] = 1.0
    return data

# print_confusion_matrix, plot_result

In [4]:
def print_confusion_matrix(self, w):
    # print w
    print("w:")
    for i in range(w.shape[0]):
        print("{:15.10f}".format(w[i, 0]))
    print()
        
    # print confusion matrix
    Xw = self.X @ w
    c1, c2 = [], []
    tp, fp, tn, fn = 0.0, 0.0, 0.0, 0.0
    cnt = 1
    for i in range(Xw.shape[0]):
        threshold = Xw[i, 0]
        label = self.Label[i, 0]
        if(threshold < 0):
            if (label == 0):
                tp += 1
            else:
                fp += 1
            c1.append(i)
        else:
            if (label == 1):
                tn += 1
            else:
                fn += 1
            c2.append(i)
    print("Confusion Matrix:")
    print("\t\tPredict cluster 1 Predict cluster 2")
    print("Is cluster 1\t\t{tp}\t\t{fn}".format(tp=tp, fn=fn))
    print("Is cluster 2\t\t{fp}\t\t{tn}\n".format(fp=fp, tn=tn))
    print("Sensitivity (Successfully predict cluster 1): {:7.5f}".format(tp / (tp + fn)))
    print("Specificity (Successfully predict cluster 2): {:7.5f}\n".format(tn / (tn + fp)))

    return c1, c2


def plot_result(self):
    print("Gradient descent:\n")
    grad_c1, grad_c2 =  print_confusion_matrix(self, self.w_grad)
    print("--------------------------------------------------")
    print("Newton's method:\n")
    newtons_c1, newtons_c2 =  print_confusion_matrix(self, self.w_newtons)
    
    N = self.N
   
    plt.subplot(131)
    plt.title("Ground truth")
    c1, c2 = self.X[:N], self.X[N:]
    plt.scatter(c1[:,0], c1[:,1], c='r')
    plt.scatter(c2[:,0], c2[:,1], c='b')

    plt.subplot(132)
    plt.title("Gradient descent")
    c1, c2 = self.X[grad_c1], self.X[grad_c2]
    plt.scatter(c1[:,0], c1[:,1], c='r')
    plt.scatter(c2[:,0], c2[:,1], c='b')

    plt.subplot(133)
    plt.title("Newton's method:")
    c1, c2 = self.X[newtons_c1], self.X[newtons_c2]
    plt.scatter(c1[:,0], c1[:,1], c='r')
    plt.scatter(c2[:,0], c2[:,1], c='b')
    plt.show()   
    return

## self-defined logistic function

In [5]:
def expit(x):
    # logistic function : 1 / (1 + exp(x))
    n = x.shape[0]
    data = np.zeros((n, 1))
    for i in range(n):
        if (x[i, 0] > 30):
            data[i, 0] = 1.0
        elif (x[i, 0] < -30):
            data[i, 0] = 0.0
        else:
            data[i, 0] = 1 / (1 + math.exp(-x[i, 0]))
    return data

## LR class

In [6]:
class LR:
    def __init__(self, N, X, Label):
        # raw data
        self.N = N
        self.X = X
        self.Label = Label

        # w learned from gradient and newton's method
        self.w_grad = np.zeros((3, 1))
        self.w_newtons = np.zeros((3, 1))

        # visualization funcs
        LR.print_confusion_matrix = print_confusion_matrix
        LR.plot_result = plot_result
    

    def gradient_descent(self):
        iter = 1e4
        w = np.zeros((3, 1))
        while(iter):
            # w' = w + dw
            # dw = ▽F = XT[expit(Xw) - y]
            dw = self.X.T @ (self.Label - expit(self.X @ w))
            if (abs(dw) < 1e-6).all():
                break
            iter -= 1
            w += dw
        self.w_grad = w
        return


    def newtons(self):
        n = self.N * 2
        w = np.zeros((3, 1))
        D = np.zeros((n, n))
        iter = 1e4
        while(iter):
            # (1) ▽F = XT[expit(Xw)-y]
            expit_Xw = expit(self.X @ w)
            df = self.X.T @ (self.Label - expit_Xw)
            
            # (2) Hf = (XT)D(X)
            #      D = e^-Xw[i]/(1+e^-Xw[i])^2 
            for i in range(n):
                D[i, i] = expit_Xw[i] @ (1 - expit_Xw[i])
            Hf = self.X.T @ D @ self.X
            
            # (3) dw = ▽F(f), if H(f) is invertible
            #        = H(f)_inv @ ▽F(f), otherwise
            dw = df
            if (det(Hf) != 0):
                dw = inv(Hf) @ df
            
            # termination
            if (abs(dw) < 1e-4).all(): 
                break
            
            # update
            w += dw
            iter -= 1
        
        self.w_newtons = w
        return

## main function

In [ ]:
if __name__ == '__main__':
    test_case1 = (['-x1', '1', '2', '-y1', '1', '2', '-x2', '10', '2', '-y2', '10', '2'])
    test_case2 = (['-x1', '1', '2', '-y1', '1', '2', '-x2', '3', '4', '-y2', '3', '4'])
    args = get_parser(test_case2)
    
    # get params
    N = args.N
    d1 = data_generator(args.x1, args.y1, N)
    d2 = data_generator(args.x2, args.y2, N)
    X = np.concatenate((d1, d2), axis=0)
    
    # Label = {0, 1}
    Label = np.zeros((2 * N, 1))
    for i in range(N, 2 * N):
        Label[i, 0] = 1
    
    # run gradient_descent() and newtons()
    LR_Model = LR(N, X, Label)
    LR_Model.gradient_descent()
    LR_Model.newtons()

    # plot result
    LR_Model.plot_result()